In [93]:
import pandas as pd
import xlrd
import os
import warnings

file_paths = ["Rocznik_2014__GR.xls", "Rocznik_2015__GR.xls", "Rocznik_2016__GR.xls", "Rocznik_2017_GR.xls", "Rocznik_2018_GR.xls", "Rocznik_2019_GR.xls",
              "Rocznik_2020_GR.xls", "Rocznik_2021_GR.xls", "Rocznik_2022_GR.xls"]

# map files to tab names [tabs are mixed between years]
files_to_tab_names = dict()
headers = dict()
for ind, path in enumerate(file_paths):
  files_to_tab_names[path] = dict()
  headers[path] = dict()

  if ind <= 3:
    files_to_tab_names[path]["wartosci_akcji"] = "Tab 13"
    files_to_tab_names[path]["najwyzsze_sesyjne_obroty"] = "Tab 14"
    files_to_tab_names[path]["stopy_zwrotu"] = "Tab 15"
    files_to_tab_names[path]["najwyzsze_stopy_zwrotu"] = "Tab 16"
  else:
    files_to_tab_names[path]["wartosci_akcji"] = "Tab 8"
    files_to_tab_names[path]["najwyzsze_sesyjne_obroty"] = "Tab 9"
    files_to_tab_names[path]["stopy_zwrotu"] = "Tab 10"
    files_to_tab_names[path]["najwyzsze_stopy_zwrotu"] = "Tab 11"

  if ind <= 7:
    headers[path]["wartosci_akcji"] = [3]
  else:
    headers[path]["wartosci_akcji"] = [3]

def read_xls_files(file_paths):

    warnings.filterwarnings("ignore")
    df_wartosci_akcji = pd.DataFrame()
    df_sesyjne_obroty = pd.DataFrame()
    df_stopy_zwrotu = pd.DataFrame()
    df_najwyzsze_stopy_zwrotu = pd.DataFrame()

    for file_path in file_paths:
        if os.path.exists(file_path):
            xls = xlrd.open_workbook(file_path)
            for sheet_name in xls.sheet_names():

                  if sheet_name == files_to_tab_names[file_path]["wartosci_akcji"]:
                    df = pd.read_excel(file_path, sheet_name=sheet_name, header=headers[path]["wartosci_akcji"], usecols = range(11))
                    df_wartosci_akcji = pd.concat([df_wartosci_akcji, df], ignore_index=True, axis = 0)

                  elif sheet_name == files_to_tab_names[file_path]["najwyzsze_sesyjne_obroty"]:
                    df = pd.read_excel(file_path, sheet_name=sheet_name, header=[2], usecols=range(5))
                    df_sesyjne_obroty = pd.concat([df_sesyjne_obroty, df], ignore_index=True, axis = 0)

                  elif sheet_name == files_to_tab_names[file_path]["stopy_zwrotu"]:
                    df = pd.read_excel(file_path, sheet_name=sheet_name, header=[2,3,4])
                    df_stopy_zwrotu = pd.concat([df_stopy_zwrotu, df], ignore_index=True, axis = 0)

                  elif sheet_name == files_to_tab_names[file_path]["najwyzsze_stopy_zwrotu"]:
                    df = pd.read_excel(file_path, sheet_name=sheet_name, header=[2], usecols=range(5))
                    df_najwyzsze_stopy_zwrotu = pd.concat([df_najwyzsze_stopy_zwrotu, df], ignore_index=True, axis = 0)

        else:
            print(f"File not found: {file_path}")

    return df_wartosci_akcji, df_sesyjne_obroty, df_stopy_zwrotu, df_najwyzsze_stopy_zwrotu

# Example usage:
df_wartosci_akcji, df_sesyjne_obroty, df_stopy_zwrotu, df_najwyzsze_stopy_zwrot = read_xls_files(file_paths)

WARNING *** file size (1183124) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (1183124) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (1183124) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (1183124) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (1183124) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [136]:
def read_excel_files(path):
    excel_data = {}
    for file in os.listdir(path):
        if file.endswith(".xls"):
            year = file.split('_')[1]  # Extracting year from file name
            excel_data[year] = {}
            xls = pd.ExcelFile(os.path.join(path, file))
            for sheet_name in xls.sheet_names:

              if sheet_name == files_to_tab_names[file]["wartosci_akcji"]:
                excel_data[year]["wartosci_akcji"] = pd.read_excel(xls, sheet_name=sheet_name, header=headers[file]["wartosci_akcji"], usecols = range(11))

              # elif sheet_name == files_to_tab_names[file]["najwyzsze_sesyjne_obroty"]:
              #   excel_data[year]["najwyzsze_sesyjne_obroty"] = pd.read_excel(file, sheet_name=sheet_name, header=[2], usecols=range(5))

              # elif sheet_name == files_to_tab_names[file]["stopy_zwrotu"]:
              #   excel_data[year]["stopy_zwrotu"] = pd.read_excel(file, sheet_name=sheet_name, header=[2,3,4])

              # elif sheet_name == files_to_tab_names[file]["najwyzsze_stopy_zwrotu"]:
              #   excel_data[year]["najwyzsze_stopy_zwrotu"] = pd.read_excel(file, sheet_name=sheet_name, header=[2], usecols=range(5))
    print(excel_data.keys())
    return excel_data
excel_data = read_excel_files("/Market_Value")

WARNING *** file size (1183124) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
dict_keys(['2020', '2016', '2022', '2018', '2015', '2014', '2021', '2017', '2019'])


In [137]:
def separate_data_by_company(excel_data):

    company_dfs = {}
    for year, sheets in excel_data.items():
        for sheet_name, df in sheets.items():
            company_col = None
            if 'Lp./ No' in df.columns:
              df.drop(columns = ['Lp./ No'])
            for col in df.columns:
                if 'Spółka/ Company' in col:
                    company_col = col
                    break
                elif 'Akcje/ Shares' in col:
                    company_col = col
                    break
                elif 'Spółka/Company' in col:
                    company_col = col
                    break
                elif 'Akcje/Shares' in col:
                    company_col = col
                    break
                elif 'Spółka / Company' in col:
                    company_col = col
                    break
                elif 'Akcje / Shares' in col:
                    company_col = col
                    break

            if company_col is None:
                raise ValueError(f"Company column not found in the DataFrame, sheet_name={sheet_name}, year = {year}")

            for company, group in df.groupby(by=company_col):
                if company not in company_dfs:
                    # Initialize DataFrame with columns corresponding to each year
                    company_dfs[company] = pd.DataFrame(columns=excel_data.keys())
                # Add data for the company and year, transposing the group
                company_dfs[company][year] = group.drop(columns=[company_col]).T
    return company_dfs
company_data = separate_data_by_company(excel_data)

In [138]:
print(company_data['06MAGNA     '].index)

Index(['Lp./ No', 'Kod isin / Isin',
       'Wartość obrotów (mln zł)/ Turnover value (PLN million)',
       'Udział w obrotach / Share in turnover (%)',
       'Roczny wskaźnik obrotu/ Annual turnover ratio (%) ',
       'Średni wolumen obrotu na sesję (akcje)/ Average volume per session (shares)',
       'Średnia liczba transakcji/ Average number of transactions per session',
       'Średnia liczba zleceń/ Average number of orders per session',
       'Unnamed: 9',
       'Wartość obrotów  (mln zł)/ Turnover value (PLN million)'],
      dtype='object')


In [139]:
print(company_data['06MAGNA     '].head())

                                                            2020 2016  \
Lp./ No                                                      472  NaN   
Kod isin / Isin                                     PLNFI0600010  NaN   
Wartość obrotów (mln zł)/ Turnover value (PLN m...           0.0  NaN   
Udział w obrotach / Share in turnover (%)                    0.0  NaN   
Roczny wskaźnik obrotu/ Annual turnover ratio (%)            0.0  NaN   

                                                            2022 2018 2015  \
Lp./ No                                                      145  NaN  NaN   
Kod isin / Isin                                     PLNFI0600010  NaN  NaN   
Wartość obrotów (mln zł)/ Turnover value (PLN m...        72.519  NaN  NaN   
Udział w obrotach / Share in turnover (%)               0.000232  NaN  NaN   
Roczny wskaźnik obrotu/ Annual turnover ratio (%)          156.8  NaN  NaN   

                                                   2014          2021 2017  \
Lp./ No       